In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib2
import sys
import random
import time

In [2]:
%matplotlib inline
pd.options.display.max_rows = 10

In [3]:
def get_awards(imdb_id):
    content = urllib2.urlopen('http://www.imdb.com/name/'+imdb_id+'/awards?ref_=nm_awd').read()
    soup1 = BeautifulSoup(content, "lxml")
    sentence = soup1.find_all('div',attrs={'class':'desc'});
    if len(sentence) == 0:
        return 0
    else:
        s = str(sentence[0]).split('wins')
        return int(''.join(c for c in s[0] if c.isdigit())) + int(''.join(c for c in s[1] if c.isdigit()))

In [ ]:
dataset = pd.DataFrame.from_csv('movielinks.csv',header=0,index_col=None)
url_list = list(dataset.imdb_url)
l_url = []
l_title = []
l_rating = []
l_metacritic = []
l_director_likes = []
l_writer_likes = []
l_actor_likes = []
l_year = []
l_title_likes = []
l_director_awards = []
l_writer_awards = []
l_actor_awards = []
N = 5000;
Nscraped = 0;
for i in range(0,N):
    try:
        url = url_list[i]
        response = requests.get(url)
        soup0 = BeautifulSoup(response.text, "lxml")
        #get title
        title = str(soup0.find_all("title")[0].string).replace('IMDb','').replace('-',' ').lstrip().rstrip()
        #get year
        ystring = str(soup0.find_all('span', attrs={"id": "titleYear"})[0]).split('year/')[1]
        year = ystring[0:4]
        #get IMDB rating
        rating = float(soup0.find_all('span', attrs={"itemprop": "ratingValue"})[0].string)
        #get metacritic score
        metascore = int(soup0.find_all("a", attrs={"href": "criticreviews?ref_=tt_ov_rt"})[0].find('span').text)
        #get movie facebook likes
        list1 = url.split('/')
        title_id = list1[list1.index("title")+1]
        facebook_url = "https://www.facebook.com/widgets/like.php?width=280&show_faces=1&layout=standard&href=http%3A%2F%2Fwww.imdb.com%2Ftitle%2F{}%2F&colorscheme=light".format(title_id)
        content = urllib2.urlopen(facebook_url).read()
        soup1 = BeautifulSoup(content, "lxml")
        sentence = soup1.find_all(id="u_0_2")[0].span.string
        title_likes = str(sentence.split(" ")[0])
        #get director likes
        temp = soup0.find_all('span', attrs={"itemprop": "director"})[0].find_all('a')
        director_link = 'http://www.imdb.com/' + temp[0]['href']
        list1 = director_link.split('/')
        temp = list1[list1.index("name")+1]
        director_id = temp.split('?')[0]
        facebook_url = "https://www.facebook.com/widgets/like.php?width=280&show_faces=1&layout=standard&href=http%3A%2F%2Fwww.imdb.com%2Fname%2F{}%2F&colorscheme=light".format(director_id)
        content = urllib2.urlopen(facebook_url).read()
        soup1 = BeautifulSoup(content, "lxml")
        sentence = soup1.find_all(id="u_0_2")[0].span.string
        director_likes = str(sentence.split(" ")[0])
        #director awards + nominations
        director_awards = get_awards(director_id)
        #get writer likes
        temp = soup0.find_all('span', attrs={"itemprop": "creator"})[0].find_all('a')
        writer_link = 'http://www.imdb.com' + temp[0]['href']
        list1 = writer_link.split('/')
        temp = list1[list1.index("name")+1]
        writer_id = temp.split('?')[0]
        facebook_url = "https://www.facebook.com/widgets/like.php?width=280&show_faces=1&layout=standard&href=http%3A%2F%2Fwww.imdb.com%2Fname%2F{}%2F&colorscheme=light".format(writer_id)
        content = urllib2.urlopen(facebook_url).read()
        soup1 = BeautifulSoup(content, "lxml")
        sentence = soup1.find_all(id="u_0_2")[0].span.string
        writer_likes = str(sentence.split(" ")[0])
        #get writer awards
        writer_awards = get_awards(writer_id)
        #get actor likes and awards
        actor_links = []
        temp = soup0.find_all('span', attrs={"itemprop": "actors"})
        for item in temp:
            actor = item.find_all('a')[0]
            actor_links.append('http://www.imdb.com/' + actor['href'])
        actor_likes = ''
        actor_awards = 0
        for item in actor_links:
            list1 = item.split('/')
            temp = list1[list1.index("name")+1]
            actor_id = temp.split('?')[0]
            facebook_url = "https://www.facebook.com/widgets/like.php?width=280&show_faces=1&layout=standard&href=http%3A%2F%2Fwww.imdb.com%2Fname%2F{}%2F&colorscheme=light".format(actor_id)
            content = urllib2.urlopen(facebook_url).read()
            soup = BeautifulSoup(content, "lxml")
            sentence = soup.find_all(id="u_0_2")[0].span.string
            actor_likes = actor_likes + str(sentence.split(" ")[0]) + ','
            actor_awards += get_awards(actor_id)
        #append to new lists
        l_url.append(url)
        l_title.append(title)
        l_year.append(year)
        l_rating.append(rating)
        l_metacritic.append(metascore)
        l_director_likes.append(director_likes)
        l_writer_likes.append(writer_likes)
        l_actor_likes.append(actor_likes)
        l_title_likes.append(title_likes)
        l_director_awards.append(director_awards)
        l_writer_awards.append(writer_awards)
        l_actor_awards.append(actor_awards)
        #print progress
        if i % 10 == 0:
            print 'Finished ' + str(i) + '/' + str(N) +', ' + str(Nscraped) + ' valid titles recorded'
            sys.stdout.flush()
        Nscraped += 1
    except:
        continue

Finished 0/5000, 0 valid titles recorded


In [ ]:
filelist = pd.DataFrame({'url' : l_url, 'title' : l_title, 'year' : l_year, 'imdb' : l_rating, 'metacritic' : l_metacritic,
                         'director' : l_director_likes, 'writer' : l_writer_likes, 
                         'actor' : l_actor_likes, 'likes' : l_title_likes, 'dir_awd' : l_director_awards,
                         'writer_awd' : l_writer_awards, 'actor_awd' : l_actor_awards})

In [ ]:
filelist.to_csv('dataset.csv')

In [ ]:
dataset = pd.DataFrame.from_csv('movielinks.csv',header=0,index_col=None)
url_list = list(dataset.imdb_url)    
response = requests.get(url_list[0])
soup0 = BeautifulSoup(response.text, "lxml")
temp = soup0.find_all('span', attrs={"itemprop": "director"})[0].find_all('a')
director_link = 'http://www.imdb.com/' + temp[0]['href']
list1 = director_link.split('/')
temp = list1[list1.index("name")+1]
director_id = temp.split('?')[0]

In [ ]:
director_id

In [ ]:
get_awards(director_id)

In [ ]:
l_actor_awards